<a href="https://colab.research.google.com/github/Nsimaar99/Nsima-Project/blob/master/nsiudacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Nsimaar99/Nsima-Project.git


Cloning into 'Nsima-Project'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 56 (delta 20), reused 20 (delta 20), pack-reused 21 (from 1)
Receiving objects: 100% (56/56), 876.06 KiB | 11.23 MiB/s, done.
Resolving deltas: 100% (22/22), done.


# Load the data

In [ ]:
url = "https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz"

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms =

# TODO: Load the datasets with ImageFolder
image_datasets =

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders =

#### Label mapping
You'll also need to load in a mapping from category label to category name. You can find this in the file cat_to_name.json. It's a JSON object which you can read in with the json module. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

#### Building and training the classifier

In [ ]:
# Load a pre-trained network (If you need a starting point, the VGG networks work great and are straightforward to use)
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
# Train the classifier layers using backpropagation using the pre-trained network to get the features
# Track the loss and accuracy on the validation set to determine the best hyperparameters

In [ ]:
# Load a pre-trained network (If you need a starting point, the VGG networks work great and are straightforward to use)
import torch
import torchvision.models as models

# Load the pretrained ResNet-18 model
model = models.resnet18(pretrained=True)

# Print the model architecture
print(model)

In [ ]:
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
from torch.nn import nn

In [ ]:
import torch

torch.nn.Linear()